In [51]:
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime

In [52]:
mnist=input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [53]:
dir(mnist)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 '_source',
 'count',
 'index',
 'test',
 'train',
 'validation']

In [54]:
n_inputs=mnist.train.images[0].shape[0]
n_hidden_1=300
n_hidden_2=300
n_hidden_3=100
n_outputs=10

Construction Phase

In [55]:
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.int64,shape=None,name='y')

In [56]:
with tf.name_scope('dnn'):
    hidden1=tf.contrib.layers.fully_connected(X,n_hidden_1,scope='hidden1')
    hidden2=tf.contrib.layers.fully_connected(hidden1,n_hidden_2,scope='hidden2')
    hidden3=tf.contrib.layers.fully_connected(hidden2,n_hidden_3,scope='hidden3')
    logits=tf.contrib.layers.fully_connected(hidden3,n_outputs,scope='outputs',
                                             activation_fn=None)

In [57]:
with tf.name_scope('loss'):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy,name='loss')

In [58]:
learning_rate=0.01

In [59]:
with tf.name_scope('train'):
    optimizer=tf.train.AdamOptimizer()
    training_op=optimizer.minimize(loss)
    loss_summary=tf.summary.scalar('softmax cross entropy',loss)

INFO:tensorflow:Summary name softmax cross entropy is illegal; using softmax_cross_entropy instead.


In [60]:
with tf.name_scope('eval'):
    correct=tf.nn.in_top_k(logits,y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [61]:
def log_dir(prefix=''):
    now=datetime.utcnow().strftime('%Y%m%d%H%M%S')
    root_logdir='tf_logs'
    if prefix:
        prefix+='-'
    name=prefix+'run-'+now
    return "{}/{}/".format(root_logdir,name)


In [62]:
logdir=log_dir('mnist_98')
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

In [63]:
init=tf.global_variables_initializer()
saver=tf.train.Saver()

Execution Phase

In [64]:
n_epochs=200
batch_size=50
checkpoint_path='/tmp/mnist_98_model.ckpt'
checkpoint_epoch_path=checkpoint_path+'.epoch'
final_model_path='./mnist_98_model'

In [65]:
with tf.Session() as sess:
    
    # check to see if existing weights from interruption
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path,'rb') as f:
            start_epoch=int(f.read())
            print('Training was interrupted. Continuing at epoch ',start_epoch)
            saver.restore(sess,checkpoint_path)
    else:
        start_epoch=0
        sess.run(init)
        
    for epoch in range(start_epoch,n_epochs): 

        for iteration in range(mnist.train.num_examples //batch_size):
            X_batch, y_batch=mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,
                                             y:mnist.test.labels})
        loss_sum=loss_summary.eval(feed_dict={X:mnist.test.images,
                                             y:mnist.test.labels})
        file_writer.add_summary(loss_sum,epoch) 
        
        
        if (epoch+1)%20==0:
            print(epoch,' Train accuracy:',acc_train,' Test accuracy:',acc_test)
            saver.save(sess,checkpoint_path)
            with open(checkpoint_epoch_path,'wb') as f:
                f.write(b'%d'%(epoch+1))
    save_path=saver.save(sess,'./mnist_98_final.ckpt')
    file_writer.close()

19  Train accuracy: 1.0  Test accuracy: 0.979
39  Train accuracy: 1.0  Test accuracy: 0.9794
59  Train accuracy: 1.0  Test accuracy: 0.9825
79  Train accuracy: 1.0  Test accuracy: 0.982
99  Train accuracy: 1.0  Test accuracy: 0.981
119  Train accuracy: 1.0  Test accuracy: 0.9794
139  Train accuracy: 1.0  Test accuracy: 0.9832
159  Train accuracy: 1.0  Test accuracy: 0.9844
179  Train accuracy: 1.0  Test accuracy: 0.9844
199  Train accuracy: 1.0  Test accuracy: 0.9844


In [66]:
#tf.reset_default_graph()